In [ ]:
import os
import tkinter.filedialog as tk
from tkinter import messagebox
from datetime import datetime
import pandas as pd
import pyodbc
import warnings
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import display
import win32com.client as client
import re
 
# Suppress pandas warning about pyodbc
warnings.filterwarnings("ignore", category=UserWarning)


In [ ]:
# --- Config ---
DB_STRUCTURE_PATH = r'C:\Users\IN10031033\Desktop\Important\Querys\Jupiter NoteBook\SqlDBStructure.xlsx'
MASTER_FILE_PATH = r'C:\Users\IN10031033\Desktop\Important\Querys\Jupiter NoteBook\Mapping.xlsx'
OUTPUT_DIR = r'C:\Users\IN10031033\Desktop\Important\Final\New Folder'


In [ ]:
import re
import pandas as pd

# --- Function to clean illegal characters ---
def clean_illegal_characters(value):
    """Remove illegal characters for Excel cells"""
    if isinstance(value, str):
        return re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x84\x86-\x9f]', '', value)
    return value

# --- Function to clean entire dataframe ---
def clean_dataframe(df):
    """Clean all string columns in a dataframe"""
    df_cleaned = df.copy()

    for col in df_cleaned.columns:
        try:
            if pd.api.types.is_object_dtype(df_cleaned[col]):
                df_cleaned[col] = df_cleaned[col].apply(clean_illegal_characters)
        except Exception as e:
            print(f"Skipping column {col} due to error: {e}")

    return df_cleaned


In [ ]:
# --- Step 1: Load Workbooks ---
master_file = load_workbook(MASTER_FILE_PATH)
if 'LogFile' in master_file.sheetnames:
    del master_file['LogFile']
if 'Raw Data' in master_file.sheetnames:
    del master_file['Raw Data']
 
log_sheet = master_file.create_sheet('LogFile')
raw_data_sheet = master_file.create_sheet('Raw Data')


In [ ]:
# Setup log headers
log_sheet.cell(row=1, column=1).value = 'Facility'
log_sheet.cell(row=1, column=2).value = 'Status'
log_sheet.cell(row=1, column=3).value = 'Row Count'
log_sheet.cell(row=1, column=4).value = 'Duration (s)'


In [ ]:
# Load database list
db_file = load_workbook(DB_STRUCTURE_PATH)
db_sheet = db_file['All Sites']


In [ ]:
# --- Step 2: Select Query File ---
query_path = tk.askopenfilename(title="Select your SQL query", filetypes=[("SQL/Text Files", "*.sql *.txt")])
report_name = os.path.basename(query_path).replace(".txt", "").replace(".sql", "")
with open(query_path, 'r') as f:
    query = f.read()


In [ ]:
# --- Step 3: Prepare DB Targets ---
targets = []
for i in range(2, db_sheet.max_row + 1):
    facility = db_sheet.cell(row=i, column=1).value
    database = db_sheet.cell(row=i, column=2).value
    server = db_sheet.cell(row=i, column=3).value
 
    if server and database:
        targets.append((i, facility, database, server))



In [ ]:
# --- Step 4: Parallel Query Function ---
def fetch_data(row_num, facility, database, server, query):
    try:
        print(f"Connecting to {database} on {server}...")
        start = datetime.now()
        conn = pyodbc.connect(f"Driver={{SQL Server}};Server={server};Database={database};Trusted_Connection=yes;")
        df = pd.read_sql(query, conn)
        conn.close()
        duration = (datetime.now() - start).total_seconds()
        
        # Clean the dataframe before returning
        df_cleaned = clean_dataframe(df)
        
        return (row_num, facility, df_cleaned, len(df_cleaned), duration, "Success")
    except Exception as e:
        return (row_num, facility, None, 0, 0, f"Error: {str(e)}")


In [ ]:
# --- Step 5: Execute in Parallel ---
combined_df = pd.DataFrame()
header_written = False
 
with ThreadPoolExecutor(max_workers=60) as executor:
    futures = [executor.submit(fetch_data, *args, query) for args in targets]
    for future in as_completed(futures):
        row_num, facility, df, row_count, duration, status = future.result()
 
        # Log results in Excel
        log_sheet.cell(row=row_num, column=1).value = facility
        log_sheet.cell(row=row_num, column=2).value = status
        log_sheet.cell(row=row_num, column=3).value = row_count
        log_sheet.cell(row=row_num, column=4).value = round(duration, 2)
 
        # Write raw data (only one header) - data is already cleaned
        if df is not None and not df.empty:
            try:
                if not header_written:
                    for row in dataframe_to_rows(df, index=False, header=True):
                        raw_data_sheet.append(row)
                    header_written = True
                else:
                    for row in dataframe_to_rows(df, index=False, header=False):
                        raw_data_sheet.append(row)
                combined_df = pd.concat([combined_df, df], ignore_index=True)
            except Exception as e:
                print(f"Error writing data for {facility}: {e}")
                # Log the error in the log sheet
                log_sheet.cell(row=row_num, column=2).value = f"Write Error: {str(e)}"


In [ ]:
# --- Step 6: Add Site VLOOKUP Column ---
print("Inserting 'Site' column with VLOOKUP formula...")
raw_data_sheet.insert_cols(2)
raw_data_sheet.cell(row=1, column=2).value = 'Site'
for row in range(2, raw_data_sheet.max_row + 1):
    formula = f'=VLOOKUP(A{row},Mapp!A:B,2,0)'
    raw_data_sheet.cell(row=row, column=2).value = formula


In [ ]:
# --- Step 7: Save Final File ---
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
output_file = os.path.join(OUTPUT_DIR, f"{report_name}_{timestamp}.xlsx")
master_file.save(output_file)
print(f"✅ File saved: {output_file}")


In [ ]:
# Optional: Preview
display(combined_df.head(10))
print(f"Combined dataframe shape: {combined_df.shape}")


In [ ]:
import winsound
import ctypes
winsound.MessageBeep()
ctypes.windll.user32.MessageBoxW(0, "Hey your Code is completed", "Task Complete", 0x40000)